# BeautyGan



This script takes images scraped from Flickr that are tagged with words realted to "beauty" (e.g. symmetry, elegance, pretty, etc) as the dataset for a GAN


*Note, portions of this notebook are based on a [notebook from Jeff Heaton](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_07_2_Keras_gan.ipynb). Part of his [course](https://github.com/jeffheaton/t81_558_deep_learning) on Deep Learning*

## Implementing DCGANs in Keras

Paper that described the type of DCGAN that we will create in this module.

* Radford, A., Metz, L., & Chintala, S. (2015). [Unsupervised representation learning with deep convolutional generative adversarial networks](https://arxiv.org/abs/1511.06434). *arXiv preprint arXiv:1511.06434*.

This paper implements a DCGAN as follows:

* No pre-processing was applied to training images besides scaling to the range of the tanh activation function [-1, 1]. 
* All models were trained with mini-batch stochastic gradient descent (SGD) with a mini-batch size of 128. 
* All weights were initialized from a zero-centered Normal distribution with standard deviation 0.02. 
* In the LeakyReLU, the slope of the leak was set to 0.2 in all models.
* we used the Adam optimizer(Kingma & Ba, 2014) with tuned hyperparameters. We found the suggested learning rate of 0.001, to be too high, using 0.0002 instead. 
* Additionally, we found leaving the momentum term $\beta{1}$ at the suggested value of 0.9 resulted in training oscillation and instability while reducing it to 0.5 helped stabilize training.

The paper also provides the following architecture guidelines for stable Deep Convolutional GANs:

* Replace any pooling layers with strided convolutions (discriminator) and fractional-strided convolutions (generator).
* Use batchnorm in both the generator and the discriminator.
* Remove fully connected hidden layers for deeper architectures.
* Use ReLU activation in generator for all layers except for the output, which uses Tanh.
* Use LeakyReLU activation in the discriminator for all layers.

While creating the material for this module I used a number of Internet resources, some of the most helpful were:

* [Deep Convolutional Generative Adversarial Network (TensorFlow 2.0 example code)](https://www.tensorflow.org/tutorials/generative/dcgan)
* [Keep Calm and train a GAN. Pitfalls and Tips on training Generative Adversarial Networks](https://medium.com/@utk.is.here/keep-calm-and-train-a-gan-pitfalls-and-tips-on-training-generative-adversarial-networks-edd529764aa9)
* [Collection of Keras implementations of Generative Adversarial Networks GANs](https://github.com/eriklindernoren/Keras-GAN)
* [dcgan-facegenerator](https://github.com/platonovsimeon/dcgan-facegenerator), [Semi-Paywalled Article by GitHub Author](https://medium.com/datadriveninvestor/generating-human-faces-with-keras-3ccd54c17f16)



This code should be run on a GPU, it will be very slow on a CPU alone.  The following code mounts your Google drive for use with Google CoLab.  If you are not using CoLab, the following code will not work.

In [2]:

try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False
    
%cd drive/My Drive/projects/GDL_code

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Note: using Google CoLab
TensorFlow 2.x selected.


In [0]:
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

## imports

The following packages will be used to implement a basic GAN system in Python/Keras.

In [0]:
import tensorflow as tf
import numpy as np
from PIL import Image
import os
import time
import matplotlib.pyplot as plt
from models.GAN import GAN
from utils.loaders import load_beauty

## configuration

These are the constants that define how the GANs will be created for this example.  The higher the resolution, the more memory that will be needed.  Higher resolution will also result in longer run times.  For Google CoLab (with GPU) 128x128 resolution is as high as can be used (due to memory).  Note that the resolution is specified as a multiple of 32.  So **GENERATE_RES** of 1 is 32, 2 is 64, etc.

To run this you will need training data.  The training data can be any collection of images.  I suggest using training data from the following two locations.  Simply unzip and combine to a common directory.  This directory should be uploaded to Google Drive (if you are using CoLab). The constant **DATA_PATH** defines where these images are stored.

The source data (faces) used in this module can be found here:

* [Kaggle Faces Data New](https://www.kaggle.com/gasgallo/faces-data-new)
* [Kaggle Lag Dataset: Dataset of faces, from more than 1k different subjects](https://www.kaggle.com/gasgallo/lag-dataset)

In [4]:
# Generation resolution - Must be square 
# Training data is also scaled to this.
# Note GENERATE_RES 4 or higher  will blow Google CoLab's memory and have not
# been tested extensivly.
GENERATE_RES = 3 # Generation resolution factor (1=32, 2=64, 3=96, 4=128, etc.)
GENERATE_SQUARE = 32 * GENERATE_RES # rows/cols (should be square)
IMAGE_CHANNELS = 3

# Configuration
DATA_PATH = '/content/drive/My Drive/projects/GDL_code/data/beauty'
EPOCHS = 500
BATCH_SIZE = 32
BUFFER_SIZE = 60000
PRINT_EVERY_N_BATCHES = 5

print(f"Will generate {GENERATE_SQUARE}px square images.")

# run params
SECTION = 'gan'
RUN_ID = '0001'
DATA_NAME = 'beauty'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' #'load' #

Will generate 96px square images.


## load & preprocess images

Next we will load and preprocess the images.  This can take awhile.  Google CoLab took around an hour to process.  Because of this we store the processed file as a binary.  This way we can simply reload the processed training data and quickly use it.  It is most efficient to only perform this operation once.  The dimensions of the image are encoded into the filename of the binary file because we need to regenerate it if these change.

The TensorFlow **Dataset** is used as it can hold and quickly shuffle the data, as well as divide it into the appropriate batch sizes for training

In [5]:
# Image set has about 10000 images. Can take over an hour for initial preprocessing.
# Because of this time needed, save a Numpy preprocessed file.
# Note, that file is large enough to cause problems for some verisons of Pickle,
# so Numpy binary files are used.
training_data = load_beauty(DATA_PATH, GENERATE_SQUARE, GENERATE_SQUARE, BATCH_SIZE, BUFFER_SIZE, IMAGE_CHANNELS)

Looking for file: /content/drive/My Drive/projects/faces/training_data_96_96.npy
Loading previous training pickle...


## architecture

The code below creates the generator and discriminator.

In [ ]:
gan = GAN(input_dim = (GENERATE_SQUARE,GENERATE_SQUARE,IMAGE_CHANNELS)
        , discriminator_conv_filters = [32,64,128,256,512]
        , discriminator_conv_kernel_size = [3,3,3,3,3]
        , discriminator_conv_strides = [2,2,2,1,1]
        , discriminator_batch_norm_momentum = 0.8
        , discriminator_activation = 'leaky_relu'
        , discriminator_dropout_rate = 0.25
        , discriminator_learning_rate = 0.000175
        , generator_initial_dense_layer_size = (4, 4, 256)
        , generator_upsample = [2,2,2,GENERATE_RES,1]
        , generator_conv_filters = [256,256,128,128,IMAGE_CHANNELS]
        , generator_conv_kernel_size = [3,3,3,3,3]
        , generator_conv_strides = [1,1,1,1,1]
        , generator_batch_norm_momentum = 0.8
        , generator_activation = 'relu'
        , generator_dropout_rate = None
        , generator_learning_rate = 0.00015
        , optimiser = 'adam'
        , z_dim = 100 # Size vector to generate images from
        , preview_rows = 5 # Preview image
        , preview_cols = 5 # Preview image
        )

if mode == 'build':
    gan.save(RUN_FOLDER)
else:
    gan.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

gan.discriminator.summary()

In [ ]:
gan.generator.summary()

## training

In [ ]:
start = time.time()

gan.train(
    training_data
    , batch_size=BATCH_SIZE
    , epochs=EPOCHS
    , run_folder=RUN_FOLDER
    , print_every_n_batches=PRINT_EVERY_N_BATCHES
)

elapsed = time.time()-start
print (f'Training time: {hms_string(elapsed)}')

In [0]:
fig = plt.figure()
plt.plot([x[0] for x in gan.d_losses], color='black', linewidth=0.25)

plt.plot([x[1] for x in gan.d_losses], color='green', linewidth=0.25)
plt.plot([x[2] for x in gan.d_losses], color='red', linewidth=0.25)
plt.plot([x[0] for x in gan.g_losses], color='orange', linewidth=0.25)

plt.xlabel('batch', fontsize=18)
plt.ylabel('loss', fontsize=16)

plt.xlim(0, 2000)
plt.ylim(0, 2)

plt.show()

In [ ]:
fig = plt.figure()
plt.plot([x[3] for x in gan.d_losses], color='black', linewidth=0.25)
plt.plot([x[4] for x in gan.d_losses], color='green', linewidth=0.25)
plt.plot([x[5] for x in gan.d_losses], color='red', linewidth=0.25)
plt.plot([x[1] for x in gan.g_losses], color='orange', linewidth=0.25)

plt.xlabel('batch', fontsize=18)
plt.ylabel('accuracy', fontsize=16)

plt.xlim(0, 2000)

plt.show()